 # Демонстрация работы татарской LLM



 В этом ноутбуке демонстрируется работа языковой модели (Small GPT-2), обученной с нуля на корпусе татарского языка (Википедии).



 Модель была обучена на ~100 МБ текстов в течение 3 эпох.

In [1]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast


 ## 1. Загрузка модели и токенизатора



 Мы используем `PreTrainedTokenizerFast`, так как токенизатор был обучен и сохранен в формате `tokenizer.json`.

In [2]:
def load_model(model_path="./tatar_gpt_model"):
    print(f"Загрузка модели из {model_path}...")
    try:
        tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
        model = GPT2LMHeadModel.from_pretrained(model_path)
        
        # Перенос на GPU если доступно
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)
        print(f"Модель успешно загружена на {device}")
        return model, tokenizer, device
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        return None, None, None

model, tokenizer, device = load_model()


Загрузка модели из ./tatar_gpt_model...
Модель успешно загружена на cuda


 ## 2. Функция генерации текста



 Настроим параметры генерации (temperature, top_k, top_p) для получения более разнообразного и естественного текста.

In [3]:
def generate_text(prompt, max_length=100, num_return_sequences=1):
    if model is None:
        return ["Модель не загружена"]
        
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_length=max_length, 
            num_return_sequences=num_return_sequences, 
            
            # Параметры семплирования
            do_sample=True, 
            top_k=50,          # Ограничиваем выбор топ-50 вероятными токенами
            top_p=0.95,        # Nucleus sampling
            temperature=0.7,   # Креативность (ниже - строже, выше - безумнее)
            repetition_penalty=1.2, # Штраф за повторы
            
            pad_token_id=tokenizer.eos_token_id
        )
    
    results = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        results.append(text)
    return results


 ## 3. Тестирование модели



 Попробуем задать модели различные начала фраз (промпты) и посмотрим, как она их продолжит.

In [ ]:
prompts = [
    "Татарстан - ул",              # География / Определение
    "Казан шәһәре",                # Столица
    "Габдулла Тукай",              # Культура
    "Бүген көн",                   # Бытовое
    "Мәктәптә укучылар",           # Образование
    "Татар телендә",               # Язык
    "Кояш чыга",                   # Природа
    "Безнең ил",                   # Патриотическое
]


for prompt in prompts:
    generated_texts = generate_text(prompt, max_length=80)
    for text in generated_texts:
        # Убираем переносы строк для красивого вывода в таблицу
        clean_text = text.replace("\n", " ")
        print(f"INPUT:  {prompt}")
        print(f"OUTPUT: {clean_text}\n")


PROMPT               | GENERATED TEXT
--------------------------------------------------------------------------------
INPUT: Татарстан - ул
OUTPUT: Татарстан - ул Татарстан Республикасының Арча районындагы авыл.Халык саны — 15 тирәсендә. Почта индексы — 422824.1920-елларда нигезләнгән.Төп милләтләр (1989 елгы җанисәп буенча): руслар.Климат уртача континенталь. Кёппен-Гейгер климатлар классификациясе буенча климатның коды: Dfb. Уртача еллык һава температурасы 4.6 °C. Татарская энциклопедия, Институт Татарской энциклопедии (ИТЭ) Академии наук РТ.Даган — Татарстан Республикасының

INPUT: Казан шәһәре
OUTPUT: Казан шәһәре () — Россия Федерациясенең Европа өлешеннән килгән һава массаларын кертми, шуңа да монда салкын Арктика һавасы да, кызу Урта Азия һава массалары да җиңел үтә. Боларның нәтиҗәсе булып, биредә һава торышының кинәт үзгәреше еш күзәтелә.Халык саны динамикасы:2002 елгы җанисәп буенча күпчелекне алып торучы халыклар: билгесезТалица шәһәр округы торак пунктларыСвердловск өлкәсе

### Проверка на адекватность
**Генерация модели:** 
Татарстан - ул Татарстан Республикасының Арча районындагы авыл.Халык саны — 15 тирәсендә. Почта индексы — 422824.1920-елларда нигезләнгән.Төп милләтләр (1989 елгы җанисәп буенча): руслар.Климат уртача континенталь. Кёппен-Гейгер климатлар классификациясе буенча климатның коды: Dfb. Уртача еллык һава температурасы 4.6 °C. Татарская энциклопедия, Институт Татарской энциклопедии (ИТЭ) Академии наук РТ.Даган — Татарстан Республикасының

**Перевод через google translate:** Татарстан — село в Арском районе Республики Татарстан. Население — около 15 человек. Почтовый индекс — 422824. Основано в 1920-х годах. Основные этнические группы (по данным переписи 1989 года): русские. Климат умеренно континентальный. Климатический код по классификации Кёппена-Гейгера: Dfb. Среднегодовая температура воздуха составляет 4,6 °C. Татарская энциклопедия, Институт Татарской энциклопедии (ИТЭ) Академии наук Республики Татарстан. Даган — село в Республике Татарстан.

---

**Генерация модели:** 
Казан шәһәре () — Россия Федерациясенең Европа өлешеннән килгән һава массаларын кертми, шуңа да монда салкын Арктика һавасы да, кызу Урта Азия һава массалары да җиңел үтә. Боларның нәтиҗәсе булып, биредә һава торышының кинәт үзгәреше еш күзәтелә.Халык саны динамикасы:2002 елгы җанисәп буенча күпчелекне алып торучы халыклар: билгесезТалица шәһәр округы торак пунктларыСвердловск өлкәсе торак пунктларыТурнина () — Свердловск өлкәсенең Алапай муниципаль берәмлеге составына керүче авыл.Климаты

**Перевод через google translate:** 
Город Казань () — не получает воздушных масс из европейской части Российской Федерации, поэтому сюда легко проходят как холодные арктические, так и горячие центральноазиатские воздушные массы. В результате здесь часто наблюдаются резкие изменения погоды. Динамика населения: По данным переписи 2002 года, большинство населения составляют: неизвестно. Населенные пункты городского округа Талица. Населенные пункты Свердловской области. Турнина () — село в Алапайском муниципальном округе Свердловской области. Климат

---

**Генерация модели:** 
Безнең ил тарихы, татар халык иҗаты һәм тарихи-сәяси характеристик жанрларының «Казан утлары» дигән темага багышланган җыентыкларда басылып чыккан. Аның әдәби иҗат иткән әсәрләре  Татарстан китап нәшриятында нәшер ителгән. «Мәдәни җомга», «Ватаным Татарстан», «Татарстан яшьләре», «Сабантуй», «Тормыш юл», «Идел буе», «Казан утлары», «Шәһри Казан», «Вакыт» журналларында басылган. Әдипнең шигъри җыентыгы

**Перевод через google translate:** 
Опубликовано в сборниках, посвященных истории нашей страны, татарскому народному искусству и историко-политическим особенностям жанра «Казаньские утлари». Его литературные произведения издаются Татарстанским книгоиздательством. Публиковалось в журналах «Культурная пятница», «Моя Родина Татарстан», «Татарстанская молодежь», «Сабантуй», «Жизненный путь», «Приволжская область», «Казаньские утлари», «Шехри Казань», «Время». Сборник стихов писателя.

### Наблюдения
1. Видно, что иногда ломается токенизация и нет пробелов:
    > керүче авыл.Климаты

    > пунктларыСвердловск

2. Модель удивительно правдоподобно имитирует википедию:
    - Перечилсяет журналы:
        > Публиковалось в журналах «Культурная пятница», «Моя Родина Татарстан», «Татарстанская молодежь», «Сабантуй», «Жизненный путь», «Приволжская область», «Казаньские утлари», «Шехри Казань», «Время». Сборник стихов писателя.
    - Накидывает разных фактов: 
        > Почтовый индекс — 422824. Основано в 1920-х годах. Основные этнические группы (по данным переписи 1989 года): русские. Климат умеренно континентальный. Климатический код по классификации Кёппена-Гейгера: Dfb. Среднегодовая температура воздуха составляет 4,6 °C. Татарская энциклопедия, Институт Татарской энциклопедии (ИТЭ) Академии наук Республики Татарстан. Даган — село в Республике Татарстан.